In [2]:
import carla, time, pygame, math, random, cv2
import numpy as np
from Common import spawn_camera, preprocess_image, find_highest_segment_midpoint_and_perpendicular, spline_cubica

In [16]:
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
client.load_world('Town05')
world = client.get_world()
spectator = world.get_spectator()

In [4]:
#macchina spawnata debug
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
spawn_point = carla.Transform(carla.Location(6.6, -41.3, 2), carla.Rotation(yaw=-90))
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
camera_retro = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=180, pitch=-40)))
camera_rigth = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=90, pitch=-40)))

In [5]:
print("Distruggendo il veicolo...")
vehicle.destroy()
print("Veicolo rimosso.")

Distruggendo il veicolo...
Veicolo rimosso.


In [6]:
#macchina spawnata tipo 1
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
spawn_point = carla.Transform(carla.Location(-1, -41.3, 2), carla.Rotation(yaw=-90))
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
camera_retro = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=180, pitch=-40)))
camera_rigth = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=90, pitch=-40)))


In [8]:
#macchina spawnata tipo 2
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
spawn_point = carla.Transform(carla.Location(0, -41.3, 1), carla.Rotation(yaw=-90))
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
camera_retro = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=180, pitch=-40)))
camera_rigth = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=90, pitch=-40)))

In [24]:
#macchina spawnata tipo 3
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
spawn_point = carla.Transform(carla.Location(-0.5, -41.3, 1), carla.Rotation(yaw=-90))
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
camera_retro = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=180, pitch=-40)))
camera_rigth = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=90, pitch=-40)))

In [25]:
video_output = np.zeros((600, 800, 4), dtype=np.uint8)
def camera_callback(image):
    global video_output
    #cv2.imwrite(f'./output/{image.frame}.png', np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)))
    video_output = np.array(np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)))

camera_retro.listen(lambda image: camera_callback(image))

In [26]:
# Configura i controlli iniziali
control = carla.VehicleControl()
control.reverse = False  # Retromarcia
control.throttle = 0.3  # Velocità moderata
control.brake = 0.0

# Passi per il parcheggio
PARKING_STEP_0_DURATION = 5
PARKING_STEP_1_DURATION = 6  # Tempo per sterzare all'indietro
PARKING_STEP_2_DURATION = 2  # Tempo per raddrizzare


control.steer = -0.8  # Sterzata a destra (positivo per destra)
start_time = time.time()
while time.time() - start_time < PARKING_STEP_0_DURATION:
    vehicle.apply_control(control)
    time.sleep(0.1)


Parcheggio completato!
Distruggendo il veicolo...
Veicolo rimosso.


In [ ]:
def proces(img):
    img = preprocess_image(img)
# Trova i pixel bianchi
    white_pixels = np.column_stack(np.where(img == 255))

# Se esistono pixel bianchi, trova il punto con la massima coordinata y
    if len(white_pixels) > 0:
        lowest_point = white_pixels[np.argmax(white_pixels[:, 0])]
        y_lowest, x_lowest = lowest_point
        print(f"Punto bianco più basso: ({x_lowest}, {y_lowest})")
    
    # Calcola le coordinate della linea inclinata di 30 gradi
        length = 450  # Lunghezza della linea (modificabile)
        angle = math.radians(30)  # Inclinazione in radianti
    
    # Punto finale della linea
        x_end = int(x_lowest + length * math.cos(angle))
        y_end = int(y_lowest - length * math.sin(angle))  # y diminuisce verso l'alto

    # Disegna la linea sull'immagine originale
        output_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)  # Converti in BGR per colori
        cv2.line(output_img, (x_lowest, y_lowest), (x_end, y_end), (255, 255, 255), thickness=8)  
        img = cv2.cvtColor(output_img, cv2.COLOR_BGR2GRAY)
        colored_image = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        contours, _ = cv2.findContours(255 - img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        height, width = img.shape
        for contour in contours:
            area = cv2.contourArea(contour)
            x, y, w, h = cv2.boundingRect(contour)
            if x > 0 and y > 0 and (x + w) < width and (y + h) < height:
                if area > 1000: 
                    print(area)
                    cv2.drawContours(colored_image, [contour], -1, (0,0,255), thickness=cv2.FILLED)

        hsv_image = cv2.cvtColor(colored_image, cv2.COLOR_BGR2HSV)

# Definisci i range per il colore rosso in formato HSV
        lower_red = np.array([0, 100, 100])
        upper_red = np.array([10, 255, 255])

# Maschera per isolare il colore rosso
        mask = cv2.inRange(hsv_image, lower_red, upper_red)
        mask = cv2.Canny(mask,100,200)

        midpoint, perp_start, perp_end = find_highest_segment_midpoint_and_perpendicular(mask)
        print(midpoint)
        cv2.circle(colored_image, midpoint, 5, (255, 0, 0), -1)
        curve_points = spline_cubica((400,600), (400,500), midpoint, perp_start)
        return curve_points

In [ ]:
curve_points = proces(video_output)

def follow_curve(vehicle, curve_points, speed=30):
    # Impostiamo la velocità di base
    throttle = 0.5
    brake = 0.0
    steering = 0.0
    
    # Iteriamo sui punti della curva
    for point in curve_points:
        # Calcoliamo la direzione tra il veicolo e il punto successivo
        dx = point[0] - vehicle.get_location().x
        dy = point[1] - vehicle.get_location().y
        angle_to_target = math.atan2(dy, dx)  # Angolo tra il veicolo e il punto
        
        # Otteniamo la differenza tra l'angolo corrente e l'angolo da percorrere
        angle_difference = angle_to_target - vehicle.get_transform().rotation.yaw
        if angle_difference > math.pi:
            angle_difference -= 2 * math.pi
        elif angle_difference < -math.pi:
            angle_difference += 2 * math.pi
        
        # Impostiamo la sterzata in base all'angolo
        steering = max(-1.0, min(1.0, angle_difference / math.pi))
        
        # Impostiamo la velocità (accelera o frena)
        if abs(angle_difference) < 0.1:  # Se l'angolo è molto piccolo, vai dritto
            throttle = 0.7
        else:  # Altrimenti riduci la velocità
            throttle = 0.3
        
        vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=steering, brake=brake, reverse=True))
        
        # Aggiorna il mondo e attendi un po'
        world.tick()
        time.sleep(0.1)


follow_curve(vehicle, curve_points)